In [4]:
import polars as pl
import openai
from pydantic import BaseModel
import os


This next cell creates an Open AI client. GitHub (wisely) prevents one from putting API keys in public repositories now, so I'm using an environment variable to store my API key. ChatGPT should be able to help you create an environment variable on your system. On MacOS you can edit your shell configuration file (`.zshrc` on my machine). On a windows machine you'll use the environment variable editor application. The API key will look something like this fake one: `sk-VPAJxeOHF7YLCLweC0fFT18lbkFRPP5Yux15Rs6FTfrf6Mxj`. 

In [5]:
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

Now let's read in the training data into a polars DF.

In [6]:
df = pl.read_csv("data/training-data.txt")

print(df)

ComputeError: found more fields than defined in 'Schema'

Consider setting 'truncate_ragged_lines=True'.

## Plan

Write code for the students that splits out the top 100 rows. Give them a really simple prompt and measure the accuracy against the data I created. 

Then ask them to improve the prompt and measure the accuracy. Once they feel good about it, do all 1000 and measure the accuracy. 

How much did it cost to test those? 

Make a substantive change to the structure. Some ideas: 

1. Add a better system prompt.
1. Give the LLM a list of categories.
1. Add "Chain of Thought" type reasoning. 

Again do 100 and measure the accuracy. Then do all 1000 and report the overall accuracy. 